# TP2 - DB Normalization and Querying

The objectives of this TP are:
1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this TP, we will use a database **`wine.db`** (available in the course's website) containing wine information related to 'production' and 'sales'. 

> Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [5]:
import sqlite3    # Python interface for SQLite databases

In [6]:
def printSchema(connection):
    # Function to print the DB schema
    # Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [7]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
Master1_customer:
  0: NV(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: QTE(NUM)
Master1_person:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
Master1_sell:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
Master1_sll:
  0: QTE(NUM)
  1: REGION(TEXT)
Master1_wine:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
Master2_person:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
Master2_sell:
  0: NB(NUM)
  1: NV(NUM)
  2: QTE(NUM)
  3: DATES(NUM)
  4: REGION(TEXT)
Master2_wine:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


From this point we will use __%%sql__ magic

In [8]:
%load_ext sql
%sql sqlite:///wine.db

/home/karine/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/karine/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


'Connected: @wine.db'

# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables **Master1** and **Master2** are in the First Normal Form (1NF).

By inspecting the content of these tables we can see that multiple tuples have NULL values.

In [9]:
%%sql SELECT *
FROM Master1
LIMIT 10;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
None,None,None,None,None,3,Six,Paul,Alsace
None,None,None,None,None,6,Marmagne,Bernard,Bourgogne
None,None,None,None,None,8,Lioger d'Harduy,Gabriel,Bourgogne
None,None,None,None,None,16,Barbin,Bernard,Bourgogne
None,None,None,None,None,17,Faiveley,Guy,Bourgogne
None,None,None,None,None,18,Tramier,Jean,Bourgogne
None,None,None,None,None,19,Dupaquier,Roger,Bourgogne
None,None,None,None,None,20,Lamy,Jean,Bourgogne
None,None,None,None,None,21,Cornu,Edmond,Bourgogne
None,None,None,None,None,26,Violot,Gilbert,Bourgogne


* Notice that Jupyter *displays* 'None' instead of 'NULL'. 
  - This is only to comply with python notation.
* To account for NULL values, your SQL queries must test explicitly for 'NULL'.

Another problem in **Master1** and **Master2** is data redundancy, for example:

In [10]:
%%sql SELECT *
FROM Master1
WHERE NV = 45;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
45,Chiroubles,None,1983,90,2,Boxler,Albert,Alsace
45,Chiroubles,None,1983,912,67,Descombes,Jean Ernest,Beaujolais
45,Chiroubles,None,1983,98,71,Chalandard,Danile,Jura
45,Chiroubles,None,1983,540,78,Michlel,Pierre Emile,Jura
45,Chiroubles,None,1983,450,86,Dumazet,Marc,Rhone


In [11]:
%%sql SELECT *
FROM Master1
ORDER BY NV DESC
LIMIT 10 ;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
107,Sancerre,14,1998,7000,107,Meunier,Laurent,Rhone
101,Saint Veran,12.5,1973,213,70,Trichard,Benoit,Beaujolais
100,Seyssel,11,1974,100,25,Valentin Bouchotte,Roger,Bourgogne
99,Cote Rotie,13,1994,100,5,Joudeat,Lucien,Bourgogne
99,Cote Rotie,13,1994,199,99,Cooperative,None,Rhone
98,Corbieres,10,1978,986,24,Guillemot,Pierre,Bourgogne
98,Corbieres,10,1978,400,35,Roulot,Guy,Bourgogne
98,Corbieres,10,1978,150,75,Loye,Camille,Jura
98,Corbieres,10,1978,890,76,Maire,Henri,Jura
98,Corbieres,10,1978,340,88,Begot,Albert,Rhone


---

Additional resource for Normalization:

https://www.youtube.com/watch?v=UrYLYV7WSHM

---

#### Exercise 1.1

Convert table **Master1** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.
  
Remember that **`wine.db`** contains information related to wine 'production' and 'sells'.

> Production <---> Wine <---> Sales

A good start point is to look for the 'Wine' attributes.

**Hint:** Look for redundant information between the master tables.

In [12]:
%%sql SELECT NV, NOM, PRENOM,QTE
FROM Master1 
ORDER BY NV DESC
LIMIT  5 ;

 * sqlite:///wine.db
Done.


NV,NOM,PRENOM,QTE
107,Meunier,Laurent,7000
101,Trichard,Benoit,213
100,Valentin Bouchotte,Roger,100
99,Joudeat,Lucien,100
99,Cooperative,None,199


#### Exercise 1.2

Convert table **Master2** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.

**Note:** For this part, consider that a wine can be bought in multiple locations and multiple times per day.

In [13]:
%%sql SELECT NV, CRU, DEGRE, MILL, REGION
FROM Master2
ORDER BY NV DESC
LIMIT 5;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,REGION
107,Sancerre,14,1998,None
101,Saint Veran,12.5,1973,None
100,Seyssel,11,1974,NORMANDIE
100,Seyssel,11,1974,GRAND EST
100,Seyssel,11,1974,PROVENCE-ALPES-CÔTE D'AZUR


In [14]:
%%sql SELECT NOM, PRENOM, TYPE, QTE, DATES, LIEU, NV
FROM Master2
ORDER BY NV DESC
LIMIT 5;

 * sqlite:///wine.db
Done.


NOM,PRENOM,TYPE,QTE,DATES,LIEU,NV
None,None,None,None,None,None,107
None,None,None,None,None,None,101
Audiberti,Jacques,petit,3,1983-02-21,CAEN,100
Moliere,None,petit,11,1982-11-05,NANCY,100
Gauthier,Theophile,petit,10,1982-11-04,CANNES,100


Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables.

A table can be created from the result of a query.

In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [15]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1;

 * sqlite:///wine.db
Done.
Done.


[]

In [16]:
print("\nContent of the database")
printSchema(conn)


Content of the database
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
Master1_customer:
  0: NV(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: QTE(NUM)
Master1_person:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
Master1_sell:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
Master1_sll:
  0: QTE(NUM)
  1: REGION(TEXT)
Master1_wine:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
Master2_person:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
Master2_sell:
  0: NB(NUM)
  1: NV(NUM)
  2: QTE(NUM)
  3: DATES(NUM)
  4: REGION(TEXT)
Master2_wine:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
dummy:
  0: DEGRE(NUM)


In [17]:
%%sql
SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.


DEGRE
None
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8


Looking into "dummy", we notice that our query includes NULL. This is not allowed if we were to use DEGRE as key for a table.

To correct this, we need to change the query to explicitly test for NULL as follows:

In [18]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1
WHERE DEGRE IS NOT NULL;

SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.
Done.
Done.


DEGRE
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8
10.7


Notice that we use `NULL` given that `None` is only used for display.

In [19]:
# Remove "dummy" table
%sql DROP TABLE IF EXISTS dummy;

 * sqlite:///wine.db
Done.


[]

#### Exercise 1.3

Create the new tables from Master1:

In [26]:
%%sql 
DROP TABLE IF EXISTS Master1_wine;
DROP TABLE IF EXISTS Master1_person;
DROP TABLE IF EXISTS Master1_sell;

CREATE TABLE Master1_wine AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER1
WHERE NV IS NOT NULL;

CREATE TABLE Master1_person AS
SELECT DISTINCT NP, NOM, PRENOM
FROM MASTER1
WHERE NP IS NOT NULL;


CREATE TABLE Master1_sell AS
SELECT DISTINCT NP, NV, QTE, REGION
FROM MASTER1
WHERE NP AND NV IS NOT NULL;



 * sqlite:///wine.db
Done.
Done.
Done.
Done.
Done.
Done.


[]

#### Exercise 1.4

Create the new tables from Master2:

In [21]:
%%sql 
DROP TABLE IF EXISTS Master2_wine;
DROP TABLE IF EXISTS Master2_person;
DROP TABLE IF EXISTS Master2_sell;

CREATE TABLE Master2_wine AS
SELECT DISTINCT NV,CRU, DEGRE, MILL
FROM MASTER2
WHERE NV IS NOT NULL;

CREATE TABLE Master2_person AS
SELECT DISTINCT NB,NOM, PRENOM, TYPE
FROM MASTER2
WHERE NB IS NOT NULL;


CREATE TABLE Master2_sell AS
SELECT DISTINCT NB,NV,QTE, DATES, REGION
FROM MASTER2
WHERE NB AND NV IS NOT NULL;


 * sqlite:///wine.db
Done.
Done.
Done.
Done.
Done.
Done.


[]

# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

**Important:**

- You MUST use the normalized tables created in previous steps.
  - The normalized tables will also be used in TP3.
- Do NOT use **Master1** and **Master2** in your queries.

#### Exercise 2.1

What are the different types of clients (buveurs) by volume of purchases?

In [22]:
%%sql
SELECT TYPE from Master2_person
GROUP BY TYPE
;


 * sqlite:///wine.db
Done.


TYPE
gros
moyen
petit


In [23]:
%%sql
SELECT sum(QTE),TYPE FROM Master2_person, Master2_sell
WHERE Master2_person.NB=Master2_sell.NB
GROUP BY TYPE;

 * sqlite:///wine.db
Done.


sum(QTE),TYPE
361,gros
697,moyen
772,petit


In [24]:
%%sql

UsageError: %%sql is a cell magic, but the cell body is empty. Did you mean the line magic %sql (single %)?


#### Exercise 2.2

What regions produce Pommard or Brouilly?

In [29]:
%%sql
SELECT DISTINCT CRU, REGION from Master1_wine, Master1_sell
WHERE Master1_wine.NV=Master1_sell.NV
AND Master1_wine.CRU in ("Pommard","Brouilly");

 * sqlite:///wine.db
Done.


CRU,REGION
Pommard,Bourgogne
Pommard,Rhone
Brouilly,Bourgogne


#### Exercise 2.3

What regions produce Pommard and Brouilly?

In [31]:
%%sql
SELECT DISTINCT REGION from Master1_wine, Master1_sell
WHERE Master1_wine.NV=Master1_sell.NV AND
Master1_wine.CRU="Pommard" 
INTERSECT
SELECT DISTINCT REGION from Master1_wine, Master1_sell
WHERE Master1_wine.NV=Master1_sell.NV AND
Master1_wine.CRU="Brouilly";

 * sqlite:///wine.db
Done.


REGION
Bourgogne


#### Exercise 2.4

Get the number of wines bught by CRU and Millésime

In [36]:
%%sql
SELECT DISTINCT CRU, MILL, QTE from Master1_wine, Master1_sell
WHERE Master1_sell.NV=Master1_wine.NV
GROUP BY Master1_wine.MILL, Master1_wine.CRU
LIMIT 10;

 * sqlite:///wine.db
Done.


CRU,MILL,QTE
Cote de Brouilly,1899,70
Auxey Duresses,1914,900
Chorey les Beaunes,1955,50
Riesling,1962,None
Coteaux de Pierevert,1967,900
Chambolle Musigny,1968,120
Chapelle Chambertin,1968,40
Cotes du Rhone,1968,13
Gewurztraminer,1968,890
Grands Echezeaux,1968,60


#### Exercise 2.5

Retrieve the wine number (NV) of wines produced by more than three producers

In [48]:
%%sql
SELECT Master1_wine.NV, COUNT(Master1_sell.NP) from Master1_sell, Master1_wine 
WHERE Master1_sell.NV=Master1_wine.NV
GROUP BY Master1_sell.NV
HAVING count(Master1_sell.NP)>3

 * sqlite:///wine.db
Done.


NV,COUNT(Master1_sell.NP)
45,5
78,5
89,4
98,5


#### Exercise 2.6

Which producers have not produced any wine?

In [38]:
%%sql
SELECT * FROM Master1_person
WHERE NP IS NOT(
SELECT NP FROM Master1_sell
WHERE QTE > 0)

 * sqlite:///wine.db
Done.


NP,NOM,PRENOM
3,Six,Paul
6,Marmagne,Bernard
8,Lioger d'Harduy,Gabriel
16,Barbin,Bernard
17,Faiveley,Guy
18,Tramier,Jean
19,Dupaquier,Roger
20,Lamy,Jean
21,Cornu,Edmond
26,Violot,Gilbert


#### Exercise 2.7

What clients (buveurs) have bought at least one wine from 1980?

In [51]:
%%sql
SELECT DISTINCT NP, NOM, PRENOM 
from Master1_person
WHERE NP IN (SELECT NP from Master1_sell )

 * sqlite:///wine.db
Done.


NP,NOM,PRENOM
1,Bohn,Rene
73,Bourguignon,Emile
5,Joudeat,Lucien
10,Fournier,Jean Claude
30,Desbois,Maurice
42,Juillot,Maurice
98,Berard,None
90,Roure,Raymond
7,Lasnier,Pierre
43,Menand,Jean Claude


#### Exercise 2.8

What clients (buveurs) have NOT bought any wine from 1980?

In [39]:
%%sql
SELECT NB, NOM, PRENOM FROM Master2_person 
WHERE NB is not (SELECT NB FROM Master2_sell WHERE NV is (SELECT NV FROM Master2_wine WHERE Mill=1980 ))
ORDER BY NB
LIMIT 10;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
1,Aristote,None
3,Aron,Raymond
4,Apollinaire,Guillaume
5,Audiberti,Jacques
6,Arrabal,Fernando
7,Anouilh,Jean
8,Aragon,Louis
9,Ajar,Emile
10,Andersen,Yann
11,Breton,Andre


#### Exercise 2.9

What clients (buveurs) have bought ONLY wines from 1980?

In [42]:
%%sql
SELECT DISTINCT NB, NOM, PRENOM FROM MASTER2_person
WHERE NB IN 
(
SELECT DISTINCT NB FROM MASTER2_sell 
    WHERE NV IN (
    SELECT NV FROM MASTER2_wine
    WHERE MILL=1980
    )
)
INTERSECT
SELECT DISTINCT NB, NOM, PRENOM FROM MASTER2_person
WHERE NB IN 
(
SELECT DISTINCT NB FROM MASTER2_sell 
    WHERE NV NOT IN 
    (SELECT DISTINCT NV FROM MASTER2_wine
    WHERE MILL!=1980
    ) 
)


 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
2,Artaud,Antonin
8,Aragon,Louis
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None
61,Mallarme,Stephane


#### Exercise 2.10

List all wines from 1980

In [41]:
%%sql
SELECT DISTINCT * FROM MASTER2_wine
WHERE MILL=1980

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
4,Mercurey,10.9,1980
16,Meursault,12.1,1980
20,Cote de Brouilly,12.1,1980
26,Chateau Corton Grancey,None,1980
28,Volnay,11,1980
43,Fleurie,11.4,1980
74,Arbois,12,1980
78,Etoile,12,1980
79,Seyssel,11,1980


#### Exercise 2.11

What are the wines from 1980 bought by NB=2?

In [53]:
%%sql
SELECT * FROM MASTER2_wine
WHERE NV IS (SELECT NV FROM MASTER2_sell WHERE NB=2)
AND
MILL=1980

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980


#### Exercise 2.12

What clients (buveurs) have bought ALL the wines from 1980?

In [52]:
%%sql 
SELECT * from MASTER2_person
WHERE NB IN(SELECT NB  from MASTER2_sell s WHERE NV IN 
             (SELECT NV from MASTER2_wine WHERE MILL=1980)
            GROUP BY NB
            HAVING COUNT(DISTINCT NV) IN 
            (SELECT COUNT(NV)
            FROM MASTER2_wine
            WHERE MILL = 1980)
           );



 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
44,Gide,Andre,petit
